In [50]:
import pandas as pd
import seaborn as sns
import new_lib as nl
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split
import acquire

import warnings
warnings.filterwarnings("ignore")

In [13]:
taxis = sns.load_dataset('taxis')

In [14]:
taxis.head()

,pickup,dropoff,passengers,distance,fare,tip,tolls,total,color,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough
0,2019-03-23 20:21:09,2019-03-23 20:27:24,1,1.60,7.0,2.15,0.0,12.95,yellow,credit card,Lenox Hill West,UN/Turtle Bay South,Manhattan,Manhattan
1,2019-03-04 16:11:55,2019-03-04 16:19:00,1,0.79,5.0,0.00,0.0,9.30,yellow,cash,Upper West Side South,Upper West Side South,Manhattan,Manhattan
2,2019-03-27 17:53:01,2019-03-27 18:00:25,1,1.37,7.5,2.36,0.0,14.16,yellow,credit card,Alphabet City,West Village,Manhattan,Manhattan
3,2019-03-10 01:23:59,2019-03-10 01:49:51,1,7.70,27.0,6.15,0.0,36.95,yellow,credit card,Hudson Sq,Yorkville West,Manhattan,Manhattan
4,2019-03-30 13:27:42,2019-03-30 13:37:14,3,2.16,9.0,1.10,0.0,13.40,yellow,credit card,Midtown East,Yorkville West,Manhattan,Manhattan


In [15]:
taxis.payment.value_counts()

credit card    4577
cash           1812
Name: payment, dtype: int64

In [16]:
taxis.color.value_counts()

yellow    5451
green      982
Name: color, dtype: int64

In [17]:
taxis.describe().T

,count,mean,std,min,25%,50%,75%,max
passengers,6433.0,1.539251,1.203768,0.0,1.00,1.00,2.00,6.00
distance,6433.0,3.024617,3.827867,0.0,0.98,1.64,3.21,36.70
fare,6433.0,13.091073,11.551804,1.0,6.50,9.50,15.00,150.00
tip,6433.0,1.979220,2.448560,0.0,0.00,1.70,2.80,33.20
tolls,6433.0,0.325273,1.415267,0.0,0.00,0.00,0.00,24.02
total,6433.0,18.517794,13.815570,1.3,10.80,14.16,20.30,174.82


In [18]:
taxis = taxis.drop(['pickup', 'dropoff', 'pickup_zone', 'dropoff_zone'], axis = 1)

In [19]:
taxis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6433 entries, 0 to 6432
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   passengers       6433 non-null   int64  
 1   distance         6433 non-null   float64
 2   fare             6433 non-null   float64
 3   tip              6433 non-null   float64
 4   tolls            6433 non-null   float64
 5   total            6433 non-null   float64
 6   color            6433 non-null   object 
 7   payment          6389 non-null   object 
 8   pickup_borough   6407 non-null   object 
 9   dropoff_borough  6388 non-null   object 
dtypes: float64(5), int64(1), object(4)
memory usage: 502.7+ KB


In [21]:
taxis.isna().sum()

passengers          0
distance            0
fare                0
tip                 0
tolls               0
total               0
color               0
payment            44
pickup_borough     26
dropoff_borough    45
dtype: int64

In [27]:
taxis = taxis.dropna()

In [28]:
taxis.isnull().sum()

passengers         0
distance           0
fare               0
tip                0
tolls              0
total              0
color              0
payment            0
pickup_borough     0
dropoff_borough    0
dtype: int64

In [31]:
dummy_df = pd.get_dummies(taxis[['color', 'payment', 'pickup_borough', 'dropoff_borough']], drop_first =True)
dummy_df

,color_yellow,payment_credit card,pickup_borough_Brooklyn,pickup_borough_Manhattan,pickup_borough_Queens,dropoff_borough_Brooklyn,dropoff_borough_Manhattan,dropoff_borough_Queens,dropoff_borough_Staten Island
0,1,1,0,1,0,0,1,0,0
1,1,0,0,1,0,0,1,0,0
2,1,1,0,1,0,0,1,0,0
3,1,1,0,1,0,0,1,0,0
4,1,1,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
6428,0,1,0,1,0,0,1,0,0
6429,0,1,0,0,1,0,0,0,0
6430,0,0,1,0,0,1,0,0,0
6431,0,1,1,0,0,1,0,0,0


In [32]:
taxis = pd.concat([taxis, dummy_df], axis = 1)
taxis

,passengers,distance,fare,tip,tolls,total,color,payment,pickup_borough,dropoff_borough,color_yellow,payment_credit card,pickup_borough_Brooklyn,pickup_borough_Manhattan,pickup_borough_Queens,dropoff_borough_Brooklyn,dropoff_borough_Manhattan,dropoff_borough_Queens,dropoff_borough_Staten Island
0,1,1.60,7.0,2.15,0.0,12.95,yellow,credit card,Manhattan,Manhattan,1,1,0,1,0,0,1,0,0
1,1,0.79,5.0,0.00,0.0,9.30,yellow,cash,Manhattan,Manhattan,1,0,0,1,0,0,1,0,0
2,1,1.37,7.5,2.36,0.0,14.16,yellow,credit card,Manhattan,Manhattan,1,1,0,1,0,0,1,0,0
3,1,7.70,27.0,6.15,0.0,36.95,yellow,credit card,Manhattan,Manhattan,1,1,0,1,0,0,1,0,0
4,3,2.16,9.0,1.10,0.0,13.40,yellow,credit card,Manhattan,Manhattan,1,1,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6428,1,0.75,4.5,1.06,0.0,6.36,green,credit card,Manhattan,Manhattan,0,1,0,1,0,0,1,0,0
6429,1,18.74,58.0,0.00,0.0,58.80,green,credit card,Queens,Bronx,0,1,0,0,1,0,0,0,0
6430,1,4.14,16.0,0.00,0.0,17.30,green,cash,Brooklyn,Brooklyn,0,0,1,0,0,1,0,0,0
6431,1,1.12,6.0,0.00,0.0,6.80,green,credit card,Brooklyn,Brooklyn,0,1,1,0,0,1,0,0,0


In [34]:
taxis = taxis.rename(columns = {'dropoff_borough_Staten Island': 'dropoff_borough_Staten_Island',
             'payment_credit card': 'payment_credit_card'})
taxis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6341 entries, 0 to 6432
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   passengers                     6341 non-null   int64  
 1   distance                       6341 non-null   float64
 2   fare                           6341 non-null   float64
 3   tip                            6341 non-null   float64
 4   tolls                          6341 non-null   float64
 5   total                          6341 non-null   float64
 6   color                          6341 non-null   object 
 7   payment                        6341 non-null   object 
 8   pickup_borough                 6341 non-null   object 
 9   dropoff_borough                6341 non-null   object 
 10  color_yellow                   6341 non-null   uint8  
 11  payment_credit_card            6341 non-null   uint8  
 12  pickup_borough_Brooklyn        6341 non-null   u

In [36]:
# num_vars = ['passengers', 'distance', 'fare', 'tip', 'total']
# Can do this way or
num_vars = taxis.select_dtypes(include=['int64', 'float64'])

In [39]:
cat_vars = taxis.select_dtypes(exclude=['int64', 'float64'])

In [56]:
train, remain = train_test_split(taxis, train_size = .7)
test, validate = train_test_split(remain, test_size = .5)

18069

84322

18088